In [2]:
import torch
import numpy as np

In [3]:
# tensor 和 numpy 对象的相互转化
# a = np.array([[1, 2, 3], [4, 5, 6]])
# b = torch.tensor(a)
# c = b.numpy()
# type(a), type(b), type(c)

In [4]:
# 只有一个元素的向量可以转化为标量
# a = torch.tensor([3.5])
# a, a.item(), float(a), int(a)

In [5]:
# # reshape的一些特性
# a = torch.arange(12)
# b = a.reshape(3, 4)
# b[:] = 2    # 修改所有的值
# a, b   # 地址没有改变，实在原地址上进行操作，但是形状又不一样

In [6]:
import sympy as sp
import numpy as np
sp.init_printing(use_unicode=True)

In [7]:
X = np.array([[3, 3], [4, 3], [1, 1]])
y = np.array([1, 1, -1])
# 通过样本数量创建相应的拉格朗日乘子
alphas = sp.Matrix([sp.symbols('alpha_0:%d' % X.shape[0])])
weights = sp.Matrix([sp.symbols('w_0:%d' % X.shape[1])])
b = sp.symbols('b')

# n = sp.Idx('i', (1, X.shape[0]))
# n, i, j = sp.symbols('n i j', integer=True)

# 构建目标函数
objective_func = np.sum([weight**2 for weight in weights]) / 2
# objective_func = sp.Sum([weight**2 for weight in weights], (i, 1, n)) / 2

# 构建约束函数
subject_to_func = sp.Matrix([1-yi*(np.sum(weights*Xi)+b) for yi, Xi in zip(y, X)])
# 构建拉格朗日函数
lagrange_func = objective_func + np.sum(alphas*subject_to_func)
# sp.diff(lagrange_func, weights[0])
diff_w = sp.Matrix(sp.diff(lagrange_func, weights))
# diff_w, lagrange_func, subject_to_func
# alphas.T
diff_b = sp.diff(lagrange_func, b)
lagrange_func
# np.sum(alphas*subject_to_func)
# sp.Matrix(sp.diff(lagrange_func, weights))
diff_w

[-3⋅α₀ - 4⋅α₁ + α₂ + w₀  -3⋅α₀ - 3⋅α₁ + α₂ + w₁]

In [8]:
alpha_st_kkts = alphas.T * subject_to_func.T
# alpha_st_kkts.shape
# a = [item for item in sp.Matrix(alpha_st_kkts)]
# a = np.array([item for item in sp.Matrix(alpha_st_kkts)]).reshape(3, 3)

alpha_st_kkts = sp.Matrix(np.diag(np.array(alphas.T * subject_to_func.T)))
# alphas*subject_to_func.T
# subject_to_func
# [alphas.T * subject_to_func.T][0].shape
# subject_to_func.T
alpha_st_kkts

⎡α₀⋅(-b - 3⋅w₀ - 3⋅w₁ + 1)⎤
⎢                         ⎥
⎢α₁⋅(-b - 4⋅w₀ - 3⋅w₁ + 1)⎥
⎢                         ⎥
⎣  α₂⋅(b + w₀ + w₁ + 1)   ⎦

In [9]:
alpha_kkts = sp.Matrix([sp.Ge(alpha, 0) for alpha in alphas])
alpha_kkts

⎡α₀ ≥ 0⎤
⎢      ⎥
⎢α₁ ≥ 0⎥
⎢      ⎥
⎣α₂ ≥ 0⎦

In [10]:
st_kkts = sp.Matrix([sp.Le(st, 0) for st in subject_to_func])
st_kkts

⎡-b - 3⋅w₀ - 3⋅w₁ + 1 ≤ 0⎤
⎢                        ⎥
⎢-b - 4⋅w₀ - 3⋅w₁ + 1 ≤ 0⎥
⎢                        ⎥
⎣  b + w₀ + w₁ + 1 ≤ 0   ⎦

In [11]:
solve = sp.solve([*diff_w, diff_b, *alpha_st_kkts], [*weights, b, *alphas])
# [*diff_w, diff_b, *alpha_st_kkts]
# [*weights, b, *alphas]
result = sp.Matrix(solve)
w = result[:, 0:weights.shape[1]]
a = result[:, 3:result.shape[1]]
b = result[:, weights.shape[1]]
# b
result
b.T

⎡           -23     ⎤
⎢-1  b  -2  ────  -2⎥
⎣            13     ⎦

In [23]:
result
# for i in b:
#     for j, k in zip(w, a):
#             # dic = {'w': i, 'b': j, 'a': k}
#             # print(dic)
#             print(j, i)
# w[3, :]
# a[3, :]
# w[i, :], b[i], a[i, :]
# sp.Matrix([*w[0, :], b[0], *a[0, :]]).T
# for i in range(b.shape[0]):
#     print(w[i, :], b[i], a[i, :])
# [*w[0, :], b[0], *a[0, :]]
# sp.Matrix([[*w[i, :], b[i], *a[i, :]] for i in range(b.shape[0])])
# temp = [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]
# [{'w': j[0], 'b': j[1], 'a': j[2]} for j in [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]]
    # dic = {'w': j[0], 'b': j[1], 'a': j[2]}
    # print(dic)
r = sp.Matrix([{'w': j[0], 'b': j[1], 'a': j[2]} for j in [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]])
t1 = sp.Matrix([*r[4]['a'], r[4]['b'], *r[4]['w']])
r[0]['a']
t1.T
result

⎡ 0     0     -1    0    0     0  ⎤
⎢                                 ⎥
⎢ 0     0     b     0    0     0  ⎥
⎢                                 ⎥
⎢ 0     1     -2   3/2   -1   1/2 ⎥
⎢                                 ⎥
⎢            -23                  ⎥
⎢6/13  4/13  ────   0   2/13  2/13⎥
⎢             13                  ⎥
⎢                                 ⎥
⎣1/2   1/2    -2   1/4   0    1/4 ⎦

In [13]:
# for i in range(alpha_kkts.shape[0]):
# if alpha_kkts[i].subs(r[0]['a']):
#     print('yes')
# alpha_kkts.shape[0]
for i in range(3):
    d = {alphas[i]: r[0]['a'][i]}
    print(d)

{alpha_0: 0}
{alpha_1: 0}
{alpha_2: 0}


In [14]:
# from sympy.plotting import plot3d
# x, y = sp.symbols('x, y')
# plot3d(x*sp.exp(-x**2 - y**2), (x, -3, 3), (y, -2, 2))
# formula = x*sp.exp(-x**2 - y**2)
# formula

In [15]:
# import sympy.stats as stat
# a = sp.Matrix(sp.symbols(' '.join([('a_%d' % i) for i in range(10)])))
# y = sum([a[m]*x**m for m in range(10)])
# sp.summation(a[0], (j, 1, i))
# sp.Matrix(sp.symbols('a_0:%d' % 10)).T